<a href="https://colab.research.google.com/github/adolfoguimaraes/inteligenciaartificial/blob/main/code/08_Prolog_Parte4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Atividade Prolog 

Esse notebook descreve uma atividade de Prolog utilizando uma base de conhecimento de diagnóstico. Vale ressaltar que essa base é puramente didática e não representa, em sua totalidade, a complexidade que é de identificar sintomas para uma determinada doenças. A proposta é que vocês entendam o processo de criar uma base de conhecimento. 

A base inicial que vamos utilizar está descrita no arquivo `base_diagnostico.pl` que pode ser encontrado no repositório do GitHub ou na pasta disponível no drive: https://drive.google.com/drive/folders/19euiVFFa32TmqR_WAE8Xer-8G273efXN. 

## Imports e métodos de Apoio

In [2]:
# Rodar essa célula apenas se tiver rodando no colab.

!sudo apt install swi-prolog
!pip install pyswip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
swi-prolog is already the newest version (7.6.4+dfsg-2ubuntu2).
0 upgraded, 0 newly installed, 0 to remove and 23 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
# Para uso no colab
dataset_path = "" 

In [4]:
# Para uso no github
dataset_path = "../datasets/kb_prolog/"

In [4]:
# Carregar a biblioteca 
from pyswip import Prolog
from pyswip import Atom

# Instanciar o objeto
prolog = Prolog()

In [73]:
def atoms_to_strings(answer):
    if isinstance(answer, dict):
        result = {}
        for k in answer.keys():
            result[k] = atoms_to_strings(answer[k])
    elif isinstance(answer, list):
        result = [atoms_to_strings(val) for val in answer]
    elif isinstance(answer, Atom):
        result = answer.value
    elif isinstance(answer, int) or isinstance(answer, str) or isinstance(answer, float):
        result = answer
    else:
        print("Unsupported result from Prolog: " + str(type(answer)) + str(answer))
    return result

In [37]:
## Método para consultar a base
def consultar_base(pergunta: str):

  """
    Esse método recebe uma pergunta com parâmetro e retorna True se assertiva 
    for verdadeira e False, caso contrário. Caso a pergunta seja com variáveis, 
    o método vai retornar uma lista de dicionários, onde a chave de cada 
    dicionário é uma variável passada. 

    :param pergunta: str
    
    :return : bool, list
  """

  result_ = prolog.query(pergunta)
  
  result_list = list(result_)

  if len(result_list) == 0: return False
  else:
    if len(result_list[0].keys()) == 0: return True
    else:
      return atoms_to_strings(result_list)


In [32]:
def consultar_base_com_lista(pergunta: str):
  result_ = prolog.query(pergunta)

  final_result = []
  for element in result_:
    list_ = atoms_to_strings(element)
    final_result.append(list_)

  return final_result

## Base de Conhecimento

A base de conhecimento do arquivo `base_diagnostico.pl` possui alguns fatos sobre sintomas e doenças e uma regra `qual_doenca` que dada uma lista de sintomas, retonar as doenças que possuem os sintomas passados como parâmetro.

```prolog
:- dynamic doenca/2.

% Sintomas
sintoma(dor_de_cabeca).
sintoma(febre).
sintoma(tosse).
sintoma(dor_de_garganta).
sintoma(dor_no_corpo).
sintoma(fadiga).
sintoma(nausea).
sintoma(diarreia).
sintoma(vomito).
sintoma(dificuldade_respiratoria).

doenca(gripe, [febre, tosse, dor_de_garganta, dor_no_corpo, fadiga]).
doenca(covid19, [febre, tosse, dor_no_corpo, dificuldade_respiratoria]).
doenca(intoxicacao_alimentar, [nausea, diarreia, vomito]).
doenca(dor_de_cabeca, [dor_de_cabeca]).

qual_doenca(Sintomas, D) :- doenca(D, ListaSintomas), subset(Sintomas, ListaSintomas).
```

O código a seguir carrega essa base. 

In [8]:
prolog.consult(dataset_path + "base_diagnostico.pl")

## Realizando consultas

Vamos testar nossa base com a regra `qual_doenca`.

In [44]:
consultar_base("qual_doenca([febre, tosse],X)")

[{'X': 'gripe'}, {'X': 'covid19'}]

Se a gente fizer uma consulta de uma lista de sintomas que não casa com a base dos dados, a máquina de inferência retornar `False`. Vamos testar no código a seguir.

In [45]:
consultar_base("qual_doenca([tosse, dor_nas_costas],X)")

False

Nesse caso, podemos atualizar a base de dados para quando a doença não existir. O código a seguir, criar uma interface utilizando Python para que o usuário digite a lista de sintomas. Caso, a lista retorne vazia, ele cria a regra que deve ser adicionada na base. Isso pode ser feito utilizando `prolog.assertz(fato)`. No código a seguir, vamos adicionar a regra diretamente no arquivo. Como estamos lendo a base toda hora, não precisaríamos do `assertz` já que ele leria a nova base no arquivo. Vamos manter o `assertz` apenas para que tenha um exemplo de como ele é utilizado.

In [47]:
# Métodos de Suporte

def string_de_busca(str_):
  return "qual_doenca([" + str_ + "], X)"

def cadastrar_doenca(doenca, sintomas):
  fato = "doenca(" + doenca + ", [" + sintomas + "])"
  prolog.assertz(fato)

  with open('base_diagnostico.pl', 'a') as f:
    f.write('\n')
    f.write(fato + '.\n')

In [54]:

lista_sintomas = input("Digite a lista de sintomas (separados por vírgula): ")

prolog.consult(dataset_path + "base_diagnostico.pl")

query_busca = string_de_busca(lista_sintomas)
return_consulta = consultar_base(query_busca)


if return_consulta:
  return_list = [d['X'] for d in return_consulta]
  print("Possíveis doenças: %s" % ', '.join(return_list))
else:
  print("Os sintomas não casam com nenhuma doença da base de conhecimento.")
  print("Deseja inserir uma nova doença à base? (S = Sim ou N = Não)?")
  resp = input()
  if resp == 'S':
    print("Inserir nova doença.")
    doenca = input("Insira a doença: ")
    cadastrar_doenca(doenca, lista_sintomas)
  else: 
    print("Fim da consulta.")


Digite a lista de sintomas (separados por vírgula): tosse,febre
Possíveis doenças: gripe, covid19


## Criando mais regras 

Como atividade de sala, vamos criar mais algumas regras: 

1 - Crie uma regra que permita verificar quais são os sintomas comuns entre duas doenças. Uma dica que pode ajudar a resolver esse problema é utilizar o método do Prolog `intersection/3` que recebe duas listas nos dois primeiros atribuos e retorna no terceiro parâmetro uma lista com o resultado da interseção. 

2 - Crie uma regra que retorna `True` se duas doenças são similares, e `False`, caso contrário. Duas doenças são similares se tiverem pelo menos dois sintomas em comum. Uma dica que pode ajudar a resolver esse problema é utilizar o método do Prolog `length/2`, que recebe como primeiro parâmetro uma lista e salva no segundo parâmtro o tamanho dessa lista. 



### Respostas

**Regra 1**

```prolog
sintomas_comuns(D1, D2, L) :- doenca(D1, L1), doenca(D2, L2), intersection(L1, L2, L).

```


**Regra 2**

```prolog
doencas_similares(D1, D2) :- sintomas_comuns(D1, D2, L), length(L, Len), Len >= 2.
```

Atualize sua base de conhecimento com as novas regras diretamente no arquivo e teste a seguir.

In [56]:
prolog.consult(dataset_path + "base_diagnostico.pl")

In [57]:
consultar_base("sintomas_comuns(gripe, covid19, X).")

[{'X': ['febre', 'tosse', 'dor_no_corpo']}]

In [58]:
consultar_base('doencas_similares(gripe,covid19)')

True

## Mais regras

Vamos criar mais regras, mas para isso, vamos utilizar uma base com mais fatos. Substitua os fatos de `sintoma/1` e `doenca/2` com os da lista a seguir. 

```prolog
% Sintomas
sintoma(dor_de_cabeca).
sintoma(febre).
sintoma(tosse).
sintoma(dor_de_garganta).
sintoma(dor_no_corpo).
sintoma(fadiga).
sintoma(nausea).
sintoma(diarreia).
sintoma(vomito).
sintoma(dificuldade_respiratoria).
sintoma(espirros).
sintoma(congestao_nasal).
sintoma(dor_nas_articulacoes).
sintoma(perda_de_olfato).
sintoma(perda_de_paladar).
sintoma(dor_abdominal).
sintoma(coceira).

doenca(gripe, [febre, tosse, dor_de_garganta, dor_no_corpo, fadiga]).
doenca(covid19, [febre, tosse, dor_no_corpo, dificuldade_respiratoria]).
doenca(intoxicacao_alimentar, [nausea, diarreia, vomito]).
doenca(dor_de_cabeca, [dor_de_cabeca]).
doenca(rinite, [espirros, congestao_nasal]).
doenca(artrite, [dor_nas_articulacoes]).
doenca(covid19_variante, [febre, tosse, dor_no_corpo, dificuldade_respiratoria, perda_de_olfato, perda_de_paladar]).
doenca(gastrite, [dor_abdominal, nausea]).
doenca(urticaria, [coceira]).
```

Para essa nova base, crie uma regra `qual_doenca2` que recebe uma lista de sintomas e retorna uma doenca como provavel para a lista de sintomas, se a lista de sintomas corresponder a 60% dos sintomas da provável doença.


### Resposta

```prolog
qual_doenca2(S, D, Z) :- doenca(D, L1), intersection(S, L1, L2), length(L1, X), length(L2, Y), Z is Y/X, Z >= 0.6.
```

In [99]:
prolog.consult(dataset_path + "base_diagnostico.pl")

In [100]:
consultar_base('qual_doenca2([tosse, febre, dor_de_garganta, fadiga], X)')

[{'X': 'gripe'}]

## Executando tudo 

Vamos criar um programa em Python que recebe uma lista de sintomas e retorna a lista de doenças baseada na segunda regra que criamos e para cada doença identificada, retornar as doenças similares. Se a doença não existir, pergunta se deseja cadastrar uma nova doença.

In [110]:

lista_sintomas = input("Digite a lista de sintomas (separados por vírgula): ")

prolog.consult(dataset_path + "base_diagnostico.pl")

query_busca = "qual_doenca2([" + lista_sintomas + "], X)"
return_consulta = consultar_base(query_busca)


if return_consulta:
  return_list = [d['X'] for d in return_consulta]
  print("Possíveis doenças: %s" % ', '.join(return_list))
  

  similares = []
  for doenca in return_list:
    doencas_similares = consultar_base("doencas_similares(" + doenca + ", X)")
    doencas_similares = [d['X'] for d in doencas_similares if d['X'] != doenca]
    similares.extend(doencas_similares)

  
  print("Doenças similares: %s" % ', '.join(similares) )

else:
  print("Os sintomas não casam com nenhuma doença da base de conhecimento.")
  print("Deseja inserir uma nova doença à base? (S = Sim ou N = Não)?")
  resp = input()
  if resp == 'S':
    print("Inserir nova doença.")
    doenca = input("Insira a doença: ")
    cadastrar_doenca(doenca, lista_sintomas)
  else: 
    print("Fim da consulta.")


Digite a lista de sintomas (separados por vírgula): febre,tosse,dor_de_garganta
Possíveis doenças: gripe
Doenças similares: covid19


## É isso 😏

Com essa atividade fechamos o material de Prolog. Agora vocês têm exemplos suficientes para criar o próprio programa com a própria base de conhecimento. 